In [1]:
import kagglehub
import pandas as pd
from pathlib import Path

path = kagglehub.dataset_download("saurabhshahane/music-dataset-1950-to-2019")

path = Path(path)
df = pd.read_csv(path / 'tcc_ceds_music.csv')

df.head()


,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28372 entries, 0 to 28371
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                28372 non-null  int64  
 1   artist_name               28372 non-null  object 
 2   track_name                28372 non-null  object 
 3   release_date              28372 non-null  int64  
 4   genre                     28372 non-null  object 
 5   lyrics                    28372 non-null  object 
 6   len                       28372 non-null  int64  
 7   dating                    28372 non-null  float64
 8   violence                  28372 non-null  float64
 9   world/life                28372 non-null  float64
 10  night/time                28372 non-null  float64
 11  shake the audience        28372 non-null  float64
 12  family/gospel             28372 non-null  float64
 13  romantic                  28372 non-null  float64
 14  commun

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

reduced_df = df.sample(frac=0.1, random_state=42) 

model_lr = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression(max_iter=10000, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(df['lyrics'], df['genre'], test_size=0.2)
model_lr.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier',
                 LogisticRegression(max_iter=10000, random_state=42))])

In [19]:
y_pred = model_lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.38


In [20]:
from sklearn.dummy import DummyClassifier

dummy_model = DummyClassifier(strategy='most_frequent')
dummy_model.fit(X_train, y_train)
dummy_y_pred = dummy_model.predict(X_test)
dummy_accuracy = accuracy_score(y_test, dummy_y_pred)
print(f'Dummy Model Accuracy: {dummy_accuracy:.2f}')

Dummy Model Accuracy: 0.24


In [25]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

target_cols = ['genre', 'release_date', 'artist_name', 'topic']

reduced_df = df.sample(frac=0.1, random_state=42).copy()

encoders = {}
y_encoded = pd.DataFrame()

for col in target_cols:
    le = LabelEncoder()
    y_encoded[col] = le.fit_transform(reduced_df[col])
    encoders[col] = le  # para decodificar depois

X_train, X_test, y_train, y_test = train_test_split(
    reduced_df['lyrics'], y_encoded, test_size=0.2, random_state=42
)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=5000)),
    ('clf', MultiOutputClassifier(LogisticRegression(max_iter=1000)))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

from sklearn.metrics import accuracy_score
for i, col in enumerate(target_cols):
    acc = accuracy_score(y_test[col], y_pred[:, i])
    print(f"Acurácia para '{col}': {acc:.4f}")


Acurácia para 'genre': 0.3011
Acurácia para 'release_date': 0.0229
Acurácia para 'artist_name': 0.0035
Acurácia para 'topic': 0.8768


In [33]:
test_lyrics = """
Jenny said when she was just five years old
There was nothing happening at all
Every time she puts on a radio
There was a nothin' goin' down at all, not at all
Then one fine mornin' she puts on a New York station
You know, she couldn't believe what she heard at all
She started shakin' to that fine fine music
You know her life was saved by rock 'n' roll
Despite all the amputations
You know you could just go out
And dance to a rock 'n' roll station
It was alright (it was alright)
Hey baby, you know it was alright (it was alright)
Jenny said
When she was just about five years old
You know why parents gonna be the death of us all
Two TV sets and two Cadillac cars
Well you know, it ain't gonna help me at all, not just a tiny bit
Then one fine mornin' she turns on a New York station
She doesn't believe what she hears at all
Ooh, she started dancin' to that fine fine music
You know her life is saved by rock 'n' roll, yeah rock 'n' roll
Despite all the computations
You could just dance
To that rock 'n' roll station
And baby it was alright (it was alright)
Hey it was alright (it was alright)
Hey here she comes now!
Jump! jump!
Jenny said when she was just about five years old
Hey, you know, there was nothing happening at all, not at all
Every time I put on the radio
You know there's nothin' goin' down at all, not at all
Then one fine mornin' she hears a New York station
She just didn't believe what she heard at all, hey, not at all
She started dancin' to that fine fine music
You know her life was saved by rock 'n' roll, yes rock 'n' roll
Despite all the computation
You know you could just dance
To a rock 'n' roll station
All right
All right
And it was alright
Well, listen to me now
And it was alright
Come on, man, listen
It was alright
It was alright
And it was alright
Alright!
It's alright
Alright!
Yeah, it's alright now
Alright!
Yeah, it's alright now
Alright!
It is all alright
Alright!
Yeah, alright now
Alright!
And it's alright right now
Alright!
And it's alright now
Alright!
Oh baby
Alright!
Oh baby
Alright!
Oh baby
Alright!
Yeah, yeah, yeah, yeah
Alright!
Now it's alright now
Alright!
Ooh, it's alright now
Alright!
Oh, alright
Alright!
Alright now
Alright!
"""

predict = pipeline.predict([test_lyrics])
decoded_labels = {}
for i, col in enumerate(target_cols):
    decoded_label = encoders[col].inverse_transform([predict[0][i]])
    decoded_labels[col] = decoded_label[0]

In [34]:
print("Predicted labels:")
for col in target_cols:
    print(f"{col}: {decoded_labels[col]}")

Predicted labels:
genre: pop
release_date: 2006
artist_name: ella fitzgerald
topic: world/life


In [35]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

prompt = f"""
Look to these details and write a very short song in the style of {decoded_labels['artist_name']} about {decoded_labels['topic']}.
The song genre is {decoded_labels['genre']} and the release date is {decoded_labels['release_date']}.
"""

# response = model.generate_content(prompt)

# print(response.text)